# Task 1

---

## Web scraping and analysis

This Jupyter notebook includes some code to get you started with web scraping. We will use a package called `BeautifulSoup` to collect the data from the web. Once you've collected your data and saved it into a local `.csv` file you should start with your analysis.

### Scraping data from Skytrax

If you visit [https://www.airlinequality.com] you can see that there is a lot of data there. For this task, we are only interested in reviews related to British Airways and the Airline itself.

If you navigate to this link: [https://www.airlinequality.com/airline-reviews/british-airways] you will see this data. Now, we can use `Python` and `BeautifulSoup` to collect all the links to the reviews and then to collect the text data on each of the individual review links.

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [2]:
base_url = "https://www.airlinequality.com/airline-reviews/british-airways"
pages = 10
page_size = 100

reviews = []

# for i in range(1, pages + 1):
for i in range(1, pages + 1):

    print(f"Scraping page {i}")

    # Create URL to collect links from paginated data
    url = f"{base_url}/page/{i}/?sortby=post_date%3ADesc&pagesize={page_size}"

    # Collect HTML data from this page
    response = requests.get(url)

    # Parse content
    content = response.content
    parsed_content = BeautifulSoup(content, 'html.parser')
    for para in parsed_content.find_all("div", {"class": "text_content"}):
        reviews.append(para.get_text())
    
    print(f"   ---> {len(reviews)} total reviews")

Scraping page 1
   ---> 100 total reviews
Scraping page 2
   ---> 200 total reviews
Scraping page 3
   ---> 300 total reviews
Scraping page 4
   ---> 400 total reviews
Scraping page 5
   ---> 500 total reviews
Scraping page 6
   ---> 600 total reviews
Scraping page 7
   ---> 700 total reviews
Scraping page 8
   ---> 800 total reviews
Scraping page 9
   ---> 900 total reviews
Scraping page 10
   ---> 1000 total reviews


In [3]:
df = pd.DataFrame()
df["reviews"] = reviews
df.head()

,reviews
0,✅ Trip Verified | Flying A380 business class ...
1,✅ Trip Verified | British Airways absolutely ...
2,✅ Trip Verified | My recent experience with B...
3,✅ Trip Verified | This is to express our disp...
4,✅ Trip Verified | I flew London to Malaga on ...


In [4]:
df.to_csv("data/BA_reviews.csv")

Congratulations! Now you have your dataset for this task! The loops above collected 1000 reviews by iterating through the paginated pages on the website. However, if you want to collect more data, try increasing the number of pages!

 The next thing that you should do is clean this data to remove any unnecessary text from each of the rows. For example, "✅ Trip Verified" can be removed from each row if it exists, as it's not relevant to what we want to investigate.

Remove HTML Tags: If your scraped data contains HTML tags, use a library like Beautiful Soup to remove them. Here's an example assuming you have a column called 'review_text':

In [5]:
from bs4 import BeautifulSoup

def remove_html_tags(text):
    soup = BeautifulSoup(text, 'html.parser')
    return soup.get_text()

df['reviews'] = df['reviews'].apply(remove_html_tags)

In [6]:
df.head()

,reviews
0,✅ Trip Verified | Flying A380 business class ...
1,✅ Trip Verified | British Airways absolutely ...
2,✅ Trip Verified | My recent experience with B...
3,✅ Trip Verified | This is to express our disp...
4,✅ Trip Verified | I flew London to Malaga on ...


Lowercasing: Convert all text to lowercase to ensure consistency in your analysis. This prevents issues where "Great" and "great" might be treated as different words.

In [7]:
df['reviews'] = df['reviews'].str.lower()

In [8]:
df.head()

,reviews
0,✅ trip verified | flying a380 business class ...
1,✅ trip verified | british airways absolutely ...
2,✅ trip verified | my recent experience with b...
3,✅ trip verified | this is to express our disp...
4,✅ trip verified | i flew london to malaga on ...


Remove Punctuation: Remove special characters and punctuation marks. You can use Python's str.translate() method in combination with string.punctuation.

In [9]:
import string

def remove_punctuation(text):
    return text.translate(str.maketrans('', '', string.punctuation))

df['reviews'] = df['reviews'].apply(remove_punctuation)

Remove Stop Words: Remove common stop words (e.g., "the," "and," "is") to focus on meaningful content. You can use libraries like NLTK or spaCy for this purpose.

In [10]:
pip install nltk

Note: you may need to restart the kernel to use updated packages.


In [15]:
# from nltk.corpus import stopwords
import nltk
nltk.download('stopwords')

stop_words = set(stopwords.words('english'))

def remove_stopwords(text):
    tokens = text.split()
    tokens = [word for word in tokens if word not in stop_words]
    return ' '.join(tokens)

df['reviews'] = df['reviews'].apply(remove_stopwords)

[nltk_data] Downloading package stopwords to /home/bhavin/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [17]:
df.head()

,reviews
0,✅ trip verified flying a380 business class ple...
1,✅ trip verified british airways absolutely car...
2,✅ trip verified recent experience british airw...
3,✅ trip verified express displeasure concerns r...
4,✅ trip verified flew london malaga 27 august c...


Remove Numeric Characters: If numeric characters are not relevant to your analysis, you can remove them.

In [19]:
df['reviews'] = df['reviews'].str.replace(r'\d+', '')

/tmp/ipykernel_50498/3985624944.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  df['reviews'] = df['reviews'].str.replace(r'\d+', '')


In [20]:
df.head()

,reviews
0,✅ trip verified flying a business class pleasu...
1,✅ trip verified british airways absolutely car...
2,✅ trip verified recent experience british airw...
3,✅ trip verified express displeasure concerns r...
4,✅ trip verified flew london malaga august clu...


Remove specific text patterns like "✅ Trip Verified" from each row in your dataset

In [27]:
pattern = r'✅ trip verified'

df['reviews'] = df['reviews'].str.replace(pattern, '')

In [28]:
df.head()

,reviews
0,flying a business class pleasure ba made disa...
1,british airways absolutely care reserved seat...
2,recent experience british airways horrendous ...
3,express displeasure concerns regarding flight...
4,flew london malaga august club europe stood ...


In [29]:
df.to_csv("data/preprocessed_data.csv")